In [1]:
import pymzml
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import glob
from tqdm import tqdm
from pathlib import Path 

In [3]:
path = 'D:/UW/massmotif/mzml/DRO_DIE_1ppm_10h_exp1_method_2.mzML'

In [2]:
def get_scans(path, ms_all = False, ms_lv = 1):
    run = pymzml.run.Reader(path)
    scans = []
    if ms_all == False:
        for scan in run:
            if scan.ms_level == ms_lv:
                scans.append(scan)
    elif ms_all == True:
        for scan in run:
            scans.append(scan)
            
    return scans

In [3]:
scans = get_scans(path, True)

NameError: name 'path' is not defined

In [40]:
scans[4].selected_precursors[0]['i']

64233.562317848206

In [4]:
def ms_qc(mzml_scans):
    # seperate ms1 and ms2
    ms1 = []
    ms2 = []
    for scan in mzml_scans:
        if scan.ms_level == 1:
            ms1.append(scan)
        elif scan.ms_level == 2:
            ms2.append(scan)
    
    # ms1 QC
    tot_ms1 = len(ms1)
    TIC = []
    for scan in ms1:
        TIC.append(scan.TIC)
        
    tic_h = max(TIC)
    tic_avg = sum(TIC) / len(TIC)
    
    #ms2 QC
    if len(ms2) != 0:
        tot_ms2 = len(ms2)
        h_response = []
        precursor_mz = []
        precursor_i = []
        for scan in ms2:
            h_response.append(scan.i.max())
            precursor_mz.append(scan.selected_precursors[0]['mz'])
            precursor_i.append(scan.selected_precursors[0]['i'])
        avg_response_h = sum(h_response) / len(h_response)
        avg_precursor_i = sum(precursor_i) / len(precursor_i)
        l_precursor_i = min(precursor_i)
        precursor_range = str(round(min(precursor_mz), 2)) + '~' + str(round(max(precursor_mz),2))
    
    result = [tot_ms1, "{:.2e}".format(tic_h), "{:.2e}".format(tic_avg), round(avg_response_h, 2), "{:.2e}".format(avg_precursor_i), round(l_precursor_i, 2), precursor_range]

    return result

In [73]:
ms_qc(scans)

[945, '2.59e+07', '8.52e+06', 5732.45, '7.87e+04', 6971.21, '100.93~690.37']

In [5]:
def batch_scans(path):
    all_files = glob.glob(path + "/*.mzML")
    scans = []
    file_list = []
    for file in tqdm(all_files):
        scan = get_scans(file, True)
        scans.append(scan)
        file_list.append(Path(file).name)
    print(file_list)
    print('Batch read finished!')
    
    return scans, file_list

In [58]:
batch_scan, file_list = batch_scans('../mzml/')

['DRO_DIE_1ppm_10h_exp1_method_1.mzML', 'DRO_DIE_1ppm_10h_exp1_method_2.mzML', 'DRO_DIE_1ppm_29h_exp1_method_1.mzML', 'DRO_DIE_1ppm_29h_exp1_method_2.mzML']
Batch read finished!


In [76]:
qc_result = []
ran_scan = len(batch_scan)
for index in tqdm(np.arange(ran_scan)):
    result = ms_qc(batch_scan[index])
    result = [file_list[index]] + result
    qc_result.append(result)
col = ['file_name', 'total_ms1_scan', 'max_tic', 'avg_tic', 'avg_ms2_max', 'avg_ms2precursor_i', 'min_ms2precursor_i', 'ms2precursor_range']
d_result = pd.DataFrame(qc_result, columns = col)

100%|██████████| 4/4 [00:01<00:00,  2.44it/s]


In [77]:
d_result

,file_name,total_ms1_scan,max_tic,avg_tic,avg_ms2_max,avg_ms2precursor_i,min_ms2precursor_i,ms2precursor_range
0,DRO_DIE_1ppm_10h_exp1_method_1.mzML,903,2.59e+07,8.84e+06,5728.36,8.00e+04,6179.54,100.93~699.52
1,DRO_DIE_1ppm_10h_exp1_method_2.mzML,945,2.59e+07,8.52e+06,5732.45,7.87e+04,6971.21,100.93~690.37
2,DRO_DIE_1ppm_29h_exp1_method_1.mzML,917,2.59e+07,8.78e+06,5672.97,7.97e+04,7753.91,100.93~691.52
3,DRO_DIE_1ppm_29h_exp1_method_2.mzML,951,2.53e+07,8.50e+06,5798.55,7.82e+04,7589.72,100.93~691.51


In [6]:
def qc_gen(path):
    batch_scan, file_list = batch_scans(path)
    print('All files read in!')
    
    qc_result = []
    ran_scan = len(batch_scan)
    print('Generating QC report...')
    for index in tqdm(np.arange(ran_scan)):
        result = ms_qc(batch_scan[index])
        
        result = [file_list[index]] + result
        qc_result.append(result)
    print('Generating dataframe...')
    col = ['file_name', 'total_ms1_scan', 'max_tic', 'avg_tic', 'avg_ms2_max', 'avg_ms2precursor_i', 'min_ms2precursor_i', 'ms2precursor_range']
    d_result = pd.DataFrame(qc_result, columns = col)
    print('Finished!')
    
    return d_result

In [7]:
qc_gen('../mzml/20200416mzml/')

100%|██████████| 6/6 [00:23<00:00,  3.89s/it]


['20200413-DRODIE 1ppm 10h exp1 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 10h exp2 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 29h exp1 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 29h exp2 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 4h exp1 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 4h exp2 autoMSMS neg.mzML']
Batch read finished!
All files read in!
Generating QC report...


100%|██████████| 6/6 [00:05<00:00,  1.17it/s]


Generating dataframe...
Finished!


,file_name,total_ms1_scan,max_tic,avg_tic,avg_ms2_max,avg_ms2precursor_i,min_ms2precursor_i,ms2precursor_range
0,20200413-DRODIE 1ppm 10h exp1 autoMSMS neg.mzML,3000,4.41e+06,1.99e+05,3729.09,6.85e+04,9654.36,200.01~499.63
1,20200413-DRODIE 1ppm 10h exp2 autoMSMS neg.mzML,2915,4.52e+06,2.09e+05,3650.83,6.86e+04,9101.39,200.17~499.63
2,20200413-DRODIE 1ppm 29h exp1 autoMSMS neg.mzML,2884,4.57e+06,2.06e+05,3496.00,6.82e+04,8048.62,200.01~499.62
3,20200413-DRODIE 1ppm 29h exp2 autoMSMS neg.mzML,2898,4.63e+06,2.10e+05,3745.36,6.96e+04,13063.84,201.01~499.63
4,20200413-DRODIE 1ppm 4h exp1 autoMSMS neg.mzML,2930,4.56e+06,2.06e+05,3558.53,6.85e+04,9287.11,200.01~499.62
5,20200413-DRODIE 1ppm 4h exp2 autoMSMS neg.mzML,2941,4.57e+06,2.08e+05,3802.99,6.83e+04,9802.40,200.01~499.63
